<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/weatherANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.15.0
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# Define the model class # 32층
class WeatherANN(tf.keras.Model):
    def __init__(self):
        super(WeatherANN, self).__init__()

        self.hid1 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop1 = tf.keras.layers.Dropout(0.2)

        self.hid2 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop2 = tf.keras.layers.Dropout(0.2)

        self.result = tf.keras.layers.Dense(4,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

    def call(self, x, training=False):  # Forward pass
        y1 = self.hid1(x)
        d1 = self.drop1(y1, training=training)
        y2 = self.hid2(d1)
        d2 = self.drop2(y2, training=training)
        logits = self.result(d2)
        sy = tf.nn.softmax(logits)
        return logits, sy

2.15.0


In [2]:
# Loss function
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Optimizer and backward pass
learningRate = 0.001
optimizer = tf.optimizers.Adagrad(learningRate)

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True)
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Accuracy function
@tf.function
def accuracy(predY, y):
    correct_predictions = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return acc

In [3]:
# 훈련 데이터 함수
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

In [4]:
from google.colab import files
uploaded = files.upload()

Saving weather_classification_data.csv to weather_classification_data (1).csv


In [5]:
df = pd.read_csv("/content/weather_classification_data (1).csv")
df.keys()
# df['Cloud Cover'].unique()
df['Cloud Cover'] = np.where(df['Cloud Cover'] == 'partly cloudy', 0,
                             np.where(df['Cloud Cover'] == 'clear', 1,
                                      np.where(df['Cloud Cover'] == 'overcast', 2,3)))
df

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,0,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,0,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,1,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,1,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,2,990.67,1,Winter,2.5,mountain,Rainy
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,Summer,1.0,mountain,Rainy
13196,-1.0,76,3.5,23.0,3,1067.23,1,Winter,6.0,coastal,Snowy
13197,30.0,77,5.5,28.0,2,1012.69,3,Autumn,9.0,coastal,Cloudy
13198,3.0,76,10.0,94.0,2,984.27,0,Winter,2.0,inland,Snowy


In [6]:
# df['Season'].unique()
df['Season'] = np.where(df['Season'] == 'Winter', 0,
                             np.where(df['Season'] == 'Spring', 1,
                                      np.where(df['Season'] == 'Summer', 2,3)))
df

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,0,1010.82,2,0,3.5,inland,Rainy
1,39.0,96,8.5,71.0,0,1011.43,7,1,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,1,1018.72,5,1,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,1,1026.25,7,1,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,2,990.67,1,0,2.5,mountain,Rainy
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,2,1.0,mountain,Rainy
13196,-1.0,76,3.5,23.0,3,1067.23,1,0,6.0,coastal,Snowy
13197,30.0,77,5.5,28.0,2,1012.69,3,3,9.0,coastal,Cloudy
13198,3.0,76,10.0,94.0,2,984.27,0,0,2.0,inland,Snowy


In [7]:
# df['Location'].unique()
df['Location'] = np.where(df['Location'] == 'inland', 0,
                             np.where(df['Location'] == 'mountain', 1,2))
df

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,0,1010.82,2,0,3.5,0,Rainy
1,39.0,96,8.5,71.0,0,1011.43,7,1,10.0,0,Cloudy
2,30.0,64,7.0,16.0,1,1018.72,5,1,5.5,1,Sunny
3,38.0,83,1.5,82.0,1,1026.25,7,1,1.0,2,Sunny
4,27.0,74,17.0,66.0,2,990.67,1,0,2.5,1,Rainy
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,2,1.0,1,Rainy
13196,-1.0,76,3.5,23.0,3,1067.23,1,0,6.0,2,Snowy
13197,30.0,77,5.5,28.0,2,1012.69,3,3,9.0,2,Cloudy
13198,3.0,76,10.0,94.0,2,984.27,0,0,2.0,0,Snowy


In [8]:
# df['Weather Type'].unique()
df['Weather Type'] = np.where(df['Weather Type'] == 'Rainy', 0,
                             np.where(df['Weather Type'] == 'Cloudy', 1,
                                      np.where(df['Weather Type'] == 'Sunny', 2, 3)))
df

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,0,1010.82,2,0,3.5,0,0
1,39.0,96,8.5,71.0,0,1011.43,7,1,10.0,0,1
2,30.0,64,7.0,16.0,1,1018.72,5,1,5.5,1,2
3,38.0,83,1.5,82.0,1,1026.25,7,1,1.0,2,2
4,27.0,74,17.0,66.0,2,990.67,1,0,2.5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,2,1.0,1,0
13196,-1.0,76,3.5,23.0,3,1067.23,1,0,6.0,2,3
13197,30.0,77,5.5,28.0,2,1012.69,3,3,9.0,2,1
13198,3.0,76,10.0,94.0,2,984.27,0,0,2.0,0,3


In [9]:
# 정규화
df.keys()
X = df[['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
       'Cloud Cover', 'Atmospheric Pressure', 'UV Index', 'Season',
       'Visibility (km)', 'Location']].values
y = df[['Weather Type']].values

X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = tf.one_hot(y_train.flatten(), 4)
y_test = tf.one_hot(y_test.flatten(), 4)

train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=X_train.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)

epochs = 30000
frequency = 150

model = WeatherANN()
y_loss = train_learning(epochs, frequency, trainDataIter, model, X_test, y_test)

  0%|          | 24/30000 [00:03<48:55, 10.21it/s]  

Epoch: 1 ===> Loss: 21.0460, Accuracy: 0.2700


  1%|          | 189/30000 [00:04<03:31, 141.17it/s]

Epoch: 151 ===> Loss: 17.6504, Accuracy: 0.2800


  1%|          | 362/30000 [00:04<01:34, 313.89it/s]

...

Epoch: 29701 ===> Loss: 2.7111, Accuracy: 0.7767


100%|█████████▉| 29883/30000 [01:41<00:00, 290.61it/s]

Epoch: 29851 ===> Loss: 3.8039, Accuracy: 0.7333


100%|██████████| 30000/30000 [01:42<00:00, 293.00it/s]

Test Accuracy: 0.7470


In [12]:
path = '/content/drive/MyDrive/세종교육/weatherANN/'
model.save(path + f"weatherAnn_tf_{tf.__version__}_version0")
model.summary()

Model: "weather_ann"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  352       
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  1056      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  132       
                                                                 
Total params: 1540 (6.02 KB)
Trainable params: 1540 (6.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model1 =tf.keras.models.load_model("/content/drive/MyDrive/세종교육/weatherANN/weatherAnn_tf_2.15.0_version0")
# print(X_test[72:73])
# predY = model.predict(X_test[72:73])
# print(int(tf.argmax(y_test[72:73][0], axis=0)))
# print(predY[1].argmax())
# predY